In [7]:
from pylab import *
%matplotlib inline

In [8]:
caffe_root = '../'  # this file should be run from {caffe_root}/examples (otherwise change this line)
import caffe
import sys

In [9]:
# run scripts from caffe root
import os
os.chdir(caffe_root)
# Download data
!data/mnist/get_mnist.sh
# Prepare data
!examples/mnist/create_mnist.sh
# back to examples
os.chdir('examples')

Downloading...
Creating lmdb...
I0127 10:25:08.957190  4066 db_lmdb.cpp:35] Opened lmdb examples/mnist/mnist_train_lmdb
I0127 10:25:08.957578  4066 convert_mnist_data.cpp:88] A total of 60000 items.
I0127 10:25:08.957610  4066 convert_mnist_data.cpp:89] Rows: 28 Cols: 28
I0127 10:25:10.110926  4066 convert_mnist_data.cpp:108] Processed 60000 files.
I0127 10:25:10.123715  4067 db_lmdb.cpp:35] Opened lmdb examples/mnist/mnist_test_lmdb
I0127 10:25:10.124053  4067 convert_mnist_data.cpp:88] A total of 10000 items.
I0127 10:25:10.124069  4067 convert_mnist_data.cpp:89] Rows: 28 Cols: 28
I0127 10:25:10.316195  4067 convert_mnist_data.cpp:108] Processed 10000 files.
Done.


In [10]:
from caffe import layers as L, params as P
import 

def lenet(lmdb, batch_size):
    # our version of LeNet: a series of linear and simple nonlinear transformations
    n = caffe.NetSpec()
    
    n.data, n.label = L.Data(batch_size=batch_size, backend=P.Data.LMDB, source=lmdb,
                             transform_param=dict(scale=1./255), ntop=2)
    
    n.conv1 = L.Convolution(n.data, kernel_size=5, num_output=20, weight_filler=dict(type='xavier'))
    n.pool1 = L.Pooling(n.conv1, kernel_size=2, stride=2, pool=P.Pooling.MAX)
    n.conv2 = L.Convolution(n.pool1, kernel_size=5, num_output=10, weight_filler=dict(type='xavier'))
    n.loss =  L.SoftmaxWithLoss(n.conv2, n.label)
    
    return n.to_proto()
    
with open('mnist/lenet_auto_train.prototxt', 'w') as f:
    f.write(str(lenet('mnist/mnist_train_lmdb', 64)))
    
with open('mnist/lenet_auto_test.prototxt', 'w') as f:
    f.write(str(lenet('mnist/mnist_test_lmdb', 100)))

In [2]:
def lenet_gen(lmdb, batch_size):
    # our version of LeNet: a series of linear and simple nonlinear transformations
    n = caffe.NetSpec()
    
    n.data, n.label = L.Data(batch_size=batch_size, backend=P.Data.LMDB, source=lmdb,
                             transform_param=dict(scale=1./255), ntop=2)
    
    n.conv1 = L.Convolution(n.data, kernel_size=5, num_output=20, weight_filler=dict(type='xavier'))
    n.pool1 = L.Pooling(n.conv1, kernel_size=2, stride=2, pool=P.Pooling.MAX)
    n.conv2 = L.Convolution(n.pool1, kernel_size=5, num_output=10, weight_filler=dict(type='xavier'))
    n.loss =  L.SoftmaxWithLoss(n.conv2, n.label)
    
    return n

In [13]:
def CheckNetSize(net):
    for li in range(len(net.layers)):  # for each layer in the net
        print("{}\t{}\t".format(li, net._layer_names[li]))  
        
    # for each input to the layer (aka "bottom") print its name and shape
    for bi in list(net._bottom_ids(li)):
        print("{} ({}) ".format(net._blob_names[bi], net.blobs[net._blob_names[bi]].data.shape), "\t")
        
    # for each output of the layer (aka "top") print its name and shape
    for bi in list(net._top_ids(li)):
        print("{} ({}) ".format(net._blob_names[bi], net.blobs[net._blob_names[bi]].data.shape)) 
  
    print("")  # end of line

In [14]:
CheckNetSize(lenet_gen('mnist/mnist_train_lmdb', 64))

KeyError: 'layers'